In [8]:
# Data Loader
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from pytorch_lightning.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import json
#from CustomDataset import * 
from Horizon_DataLoader import * 
import sys
sys.path.append('../')
from config import *
from file_helper import *
#from Horizon_and_SAM.Horizon import PE_helper
from  PE_helper import *

from pytorch_lightning.callbacks import ModelCheckpoint , Callback

def collate_fn(batch):
    return tuple(zip(*batch))
#=================================
#             Augmentation
#=================================

class CustomDataModule(pl.LightningDataModule):
    def __init__(self ,
                 train_dir ,
                 test_dir , batch_size = 2,
                 num_workers = 0 , img_size=[IMG_WIDTH, IMG_HEIGHT] , use_aug = True ,padding_count = 24 ,c =0.1
                   ):
        super().__init__()
        self.train_dir = train_dir
        self.test_dir = test_dir
        
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.img_size = img_size      
        self.use_aug = use_aug
        self.padding_count  = padding_count
        self.c = c
        

        pass

    def prepare_data(self) -> None:
        # Download dataset
        pass

    def setup(self, stage):
        # Create dataset...          
                
        self.entire_dataset = CustomDataset(self.train_dir  , use_aug= self.use_aug , padding_count= self.padding_count , c=self.c , img_size=self.img_size)
        self.train_ds , self.val_ds = random_split(self.entire_dataset , [0.9, 0.1])        
        self.test_ds = CustomDataset(self.test_dir  , use_aug= False , img_size=self.img_size ,  padding_count= self.padding_count )
        
        print("image size ",self.img_size)
        pass

    # ToDo: Reture Dataloader...
    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(self.train_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=True)
    
    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.val_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=False)
    
    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_ds , batch_size= self.batch_size , num_workers= self.num_workers , shuffle=True)

    pass


# Test
dm = CustomDataModule ( train_dir= f"../anno/test_visiable_10_no_cross.json" ,
                       test_dir= f"../anno/test_visiable_10_no_cross.json" , padding_count=1024, img_size=[1024,512]
                       )


In [6]:
from raw_model_pixelwize_bbox import *
from Horizon_DataLoader import  * 
import pytorch_lightning as pl
import torch
import torch.nn.functional as F

class Model(pl.LightningModule):    
    def __init__(self):
        super().__init__()
        self.model = HorizonNet()
        
    def forward(self , x):
        prob , boxs  = self.model(x)
        print("prob" , prob.shape) # [ b , 1024 , 1]
        print("boxs" , boxs.shape) # [ b , 1024 , 5]

		return prob , boxs
        pass

	def training_step(self , input_b ,batch_idx ):
		img = input_b['image']        
		gt_prob = input_b['u_grad']
		out_prob , out_boxs = self.forward(img)

		# ToDo: Encode GT
		encode_gt = ...

		# ToDo: calculate loss
		cls_loss = F.binary_cross_entropy_with_logits(out_prob , gt_prob)
		box_loss = F.l1_loss(out_boxs , encode_gt )
		total_loss = cls_loss + box_loss
		return total_loss
	
	def validation_step(self, input_b, batch_idx):
		#ToDo...
		pass
	
	def configure_optimizers(self):
        #opt = optim.Adam(self.parameters() , lr=0.00035)
		#ToDo: Look at paper 
		# Ref: https://github.com/sunset1995/HorizonNet/blob/master/train.py#L44

        return [opt] , []

dm = CustomDataModule ( train_dir= f"../anno/train_visiable_20_no_cross.json" ,
                        test_dir= f"../anno/test_visiable_10_no_cross.json" ,
                        #test_dir= f"../anno/train_visiable_20_no_cross.json" ,
                          padding_count=85 , use_aug=False , c= 0.65,batch_size=1,
                          img_size=img_size
                       )


e:\Projects\Layout\Horizon_and_SAM\Horizon
e:\Projects\Layout\Horizon_and_SAM\Horizon
